# Wildmeshing tutorial

This is a jupyter notebook. The "real" notebook can be found [here](https://github.com/wildmeshing/wildmeshing-python/blob/master/examples/tutorial.ipynb).

To use it just import it!

In [ ]:
import wildmeshing as wm

We also import meshplot to visualize

In [ ]:
import meshplot as mp

## Triangulation

The 2D meshing supports high order


In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg")

Plot the resulting mesh

In [ ]:
mp.plot(V, F, shading={"wireframe": True})

The resulting mesh with the high-order nodes:

In [ ]:
p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p

The previous mesh also outside triangles, to avoid this we enable the option `cut_outside`

In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg", cut_outside=True)

In [ ]:
p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p

I we want to carve out the window we need to speficy a point inside

In [ ]:
V, F, nodes, F_nodes = wm.triangulate_svg("rocket.svg", cut_outside=True, hole_pts=[[255, 165]])

In [ ]:
import numpy as np

p = mp.plot(V, F, return_plot=True, shading={"wireframe": True})
p.add_points(nodes, shading={"point_size": 10})
p.add_points(np.array([255, 165]), shading={"point_size": 30, "point_color": "green"})
p

## Tetrahedralization (alpha)

The tetrahedralization part is still under developement, for the moment it supports only files as input and output. 

In [ ]:
tetra = wm.Tetrahedralizer()
tetra.load_mesh("small.stl")
tetra.tetrahedralize()
V, T = tetra.get_tet_mesh()

mp.plot(V, T)

### Boolean operations

Tetwild support boolean operation between meshes!

In [ ]:
import igl
import json

We need json for passing the CSG tree and igl to load the data

In [ ]:
v0, f0 = igl.read_triangle_mesh('csg_input/ball.stl')
v1, f1 = igl.read_triangle_mesh('csg_input/box.stl')
v2, f2 = igl.read_triangle_mesh('csg_input/x.stl')
v3, f3 = igl.read_triangle_mesh('csg_input/y.stl')
v4, f4 = igl.read_triangle_mesh('csg_input/z.stl')

We load a couple of meshes to to get classical CGR tree example
![](https://upload.wikimedia.org/wikipedia/commons/8/8b/Csg_tree.png)

We create a `Tetrahedralizer`, load all meshes, and tetrahedralize.

**Note 1** we set the `stop_quality` to 100 since we dont care to have high-quality meshes

**Note 2** the order of the 5 meshes matters, first mesh will have id 0 and so on

In [ ]:
tetra = wm.Tetrahedralizer(stop_quality=100)
tetra.set_meshes([v0, v1, v2, v3, v4], [f0, f1, f2, f3, f4])
tetra.tetrahedralize()

The mesh has been generated!


We can do the union between the last two meshes (id 3, and 4).
It is fast, since it dosent require to regenerate the tet mesh

In [ ]:
V, T = tetra.get_tet_mesh_from_csg(json.dumps({"operation":"union", "left": 3, "right": 4}))
mp.plot(V, T)

Now we create the cross by taking 2 unions.

In [ ]:
V, T = tetra.get_tet_mesh_from_csg(json.dumps({
    "operation": "union",
    "left": 4,
    "right": {
        "operation": "union",
        "left": 3,
        "right": 2
    }}))
mp.plot(V, T)

We evaluate the second part of the tree with an intersection between the first and second mesh (id 0 and 1)

In [ ]:
V, T = tetra.get_tet_mesh_from_csg(json.dumps({"operation":"intersection", "left": 0, "right": 1}))
mp.plot(V, T)

We can get the final result by a final difference

In [ ]:
V, T = tetra.get_tet_mesh_from_csg(json.dumps({
    "operation": "difference",
    "left": {
        "operation":"intersection",
        "left": 0,
        "right": 1
    },
    "right": {
        "operation": "union",
        "left": 4,
        "right": {
            "operation": "union",
            "left": 3,
            "right": 2
        }
    }
}))
mp.plot(V, T)